In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

In [2]:
# suppress warnings
warnings.filterwarnings('ignore')   

#### Data

In [3]:
# train data
df_train = pd.read_csv('Train.csv')
df_train.head(2)

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.0,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.6,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652


In [4]:
# test data
df_test = pd.read_csv('Test.csv')
df_test.head(2)

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,0OS9LVX X 2020-01-02,2020-01-02,0OS9LVX,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,...,1.445658,-95.984984,22.942019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0OS9LVX X 2020-01-03,2020-01-03,0OS9LVX,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,...,34.641758,-95.014908,18.539116,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Characteristics

In [5]:
# Filter columns with missing values
columns_with_missing_values_train = df_train.columns[df_train.isnull().any()]
columns_with_missing_values_test = df_train.columns[df_train.isnull().any()]

In [6]:
# Fill missing values in float-type columns with their mean
for column in columns_with_missing_values_train:
    if df_train[column].dtype == 'float64':  # Check if column is float-type
        df_train[column].fillna(df_train[column].mean(), inplace=True)

In [7]:
for column in columns_with_missing_values_test:
    if df_test[column].dtype == 'float64':  # Check if column is float-type
        df_test[column].fillna(df_test[column].mean(), inplace=True)

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [9]:
# Data Preprocessing
place_id = df_train['Place_ID']

# Separate features and target variable
X_train = df_train.drop(columns=[
    'target', 'Place_ID X Date', 'Place_ID', 
    'target_min', 'target_max', 'target_variance']
                       )
y_train = df_train['target']

In [10]:
# Separate features for test data
X_test = df_test.drop(columns=[
    'Place_ID X Date', 
    'Place_ID'])

In [11]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), X_train.select_dtypes(include=['float64']).columns)
    ])

In [12]:
# Define the model
linreg_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

In [13]:
# Train the model
linreg_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  Index(['precipitable_water_entire_atmosphere',
       'relative_humidity_2m_above_ground',
       'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground',
       'L3_NO2_NO2_column_number_density',
       'L...
       'L3_SO2_solar_azimuth_angle', 'L3_SO2_solar_zenith_angle',
       'L3_CH4_CH4_column_volume_mixing_ratio_dry_air',
       'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth',
       'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle',
       'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle'],
      dtype='object'))])),
                ('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [14]:
# Predict on the test set
y_pred_test = linreg_model.predict(X_test)

In [15]:
# Prepare submission file
#submission_df = pd.read_csv('SampleSubmission.csv')
#submission_df['target'] = y_pred_test
#submission_df.to_csv('submission.csv', index=False)
#print('submission csv successfully downloaded')

In [16]:
# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the model
linreg_model.fit(X_train_split, y_train_split)

# Predict on the validation set
y_pred_val = linreg_model.predict(X_val_split)

# Evaluate the model
mse = mean_squared_error(y_val_split, y_pred_val)
print("Mean Squared Error on Validation Set:", mse)

# Check MSE and print corresponding message
if mse < 1000:
    print("Model is Great")
elif 1000 <= mse < 2000:
    print("Model is Good")
elif 2000 <= mse < 3000:
    print("Model is Average")
elif 3000 <= mse < 4000:
    print("Model is Bad")
else:
    print("Model is Terrible")

Mean Squared Error on Validation Set: 1399.976889748849
Model is Good


In [17]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [18]:
# Define the models
models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge Regression', Ridge()),
    ('Lasso Regression', Lasso()),
    ('SVR', SVR()),
    ('Random Forest Regressor', RandomForestRegressor(random_state=42)),
    ('Gradient Boosting Regressor', GradientBoostingRegressor(random_state=42)),
    ('KNN Regressor', KNeighborsRegressor()),
    ('ElasticNet Regression', ElasticNet()),
    ('XGBoost Regressor', XGBRegressor(random_state=42)),
    ('LightGBM Regressor', LGBMRegressor(random_state=42)),
    ('CatBoost Regressor', CatBoostRegressor(random_state=42, verbose=False))
]

In [19]:
# Loop through each model
for name, model in models:
    print(f"Training {name}...")
    
    # Define the pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])

    # Train the model
    model_pipeline.fit(X_train_split, y_train_split)

    # Predict on the validation set
    y_pred_val = model_pipeline.predict(X_val_split)

    # Evaluate the model
    mse = mean_squared_error(y_val_split, y_pred_val)
    print(f"Mean Squared Error on Validation Set ({name}): {mse}")

    # Check MSE and print corresponding message
    if mse < 1000:
        print("Model is Great")
    elif 1000 <= mse < 2000:
        print("Model is Good")
    elif 2000 <= mse < 3000:
        print("Model is Average")
    elif 3000 <= mse < 4000:
        print("Model is Bad")
    else:
        print("Model is Terrible")

Training Linear Regression...
Mean Squared Error on Validation Set (Linear Regression): 1399.976889748849
Model is Good
Training Ridge Regression...
Mean Squared Error on Validation Set (Ridge Regression): 1401.5408748549776
Model is Good
Training Lasso Regression...
Mean Squared Error on Validation Set (Lasso Regression): 1464.3267441599326
Model is Good
Training SVR...
Mean Squared Error on Validation Set (SVR): 1498.0248894080335
Model is Good
Training Random Forest Regressor...
Mean Squared Error on Validation Set (Random Forest Regressor): 912.2646833727094
Model is Great
Training Gradient Boosting Regressor...
Mean Squared Error on Validation Set (Gradient Boosting Regressor): 1144.1374009687793
Model is Good
Training KNN Regressor...
Mean Squared Error on Validation Set (KNN Regressor): 1252.989752814136
Model is Good
Training ElasticNet Regression...
Mean Squared Error on Validation Set (ElasticNet Regression): 1527.1069412626646
Model is Good
Training XGBoost Regressor...
Mean